Loading necessary libraries

In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

from tqdm import tqdm
import pandas  as pd
import xgboost as xgb

In [ ]:
train_data = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
test_data  = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')
train_data.head()

In [ ]:
features = ['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7',
       'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14']

X_train = train_data[features]
y_train = train_data["target"]
final_X_test = test_data[features]

Splitting the training data

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_train, y_train, test_size=0.1)

# Training XGBoostRegressor

Parameters adapted from this great notebook, 
https://www.kaggle.com/hamzaghanmi/xgboost-hyperparameter-tuning-using-optuna

In [ ]:
Best_trial = {'lambda': 0.0030282073258141168, 'alpha': 0.01563845128469084, 'colsample_bytree': 0.5,
             'subsample': 0.7,'n_estimators': 4000, 'learning_rate': 0.01,'max_depth': 15,
             'random_state': 2020, 'min_child_weight': 257,'tree_method':'gpu_hist'
             ,'predictor': 'gpu_predictor'}

In [ ]:
regressor = xgb.XGBRegressor(**Best_trial)

regressor.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_test, y_test)],verbose=False)

In [ ]:
y_xgb_pred = regressor.predict(final_X_test)

In [ ]:
output1 = pd.DataFrame({"id":test_data.id, "target":y_xgb_pred})
output1.to_csv('xgb.csv', index=False)

# LightGBM Model

parameters from this awesome kernal 
https://www.kaggle.com/hamditarek/tabular-playground-series-xgboost-lightgbm

In [ ]:
params ={'random_state': 33,'n_estimators':5000,
 'min_data_per_group': 5,
 'boosting_type': 'gbdt',
 'device_type' : 'gpu',
 'num_leaves': 256,
 'num_iterations' : 5000,
 'max_dept': -1,
 'learning_rate': 0.005,
 'subsample_for_bin': 200000,
 'lambda_l1': 1.074622455507616e-05,
 'lambda_l2': 2.0521330798729704e-06,
 'n_jobs': -1,
 'cat_smooth': 1.0,
 'silent': True,
 'importance_type': 'split',
 'metric': 'rmse',
 'feature_pre_filter': False,
 'bagging_fraction': 0.8206341150202605,
 'min_data_in_leaf': 100,
 'min_sum_hessian_in_leaf': 0.001,
 'bagging_freq': 6,
 'feature_fraction': 0.5,
 'min_gain_to_split': 0.0,
 'min_child_samples': 20}

In [ ]:
from lightgbm import LGBMRegressor

lgb_model = LGBMRegressor(**params)
lgb_model.fit(X_train, y_train, eval_set=(X_test, y_test),early_stopping_rounds = 50, verbose = 0)

In [ ]:
y_pred_lgb = lgb_model.predict(final_X_test)

In [ ]:
output2 = pd.DataFrame({"id":test_data.id, "target":y_pred_lgb})
output2.to_csv('lgbm.csv', index=False)

# CatBoost Model

Parameters from this awesome notebook
https://www.kaggle.com/hamzaghanmi/xgboost-hyperparameter-tuning-using-optuna#Catboost-Using-Optuna

In [ ]:
Best_trial = {'l2_leaf_reg': 0.02247766515106271, 'max_bin': 364, 'subsample': 0.6708650091202213,
             'learning_rate': 0.010290546311954876, 'max_depth': 10, 'random_state': 24, 'min_data_in_leaf': 300,
            'loss_function': 'RMSE','n_estimators':  25000,'rsm':0.5}

In [ ]:
from catboost import CatBoostRegressor
cb_model = CatBoostRegressor(**Best_trial)

In [ ]:
cb_model.fit(X_train, y_train,eval_set=(X_test, y_test),use_best_model=True,verbose=1000)

In [ ]:
cat_pred =cb_model.predict(final_X_test)

In [ ]:
output3 = pd.DataFrame({"id":test_data.id, "target":cat_pred})
output3.to_csv('cat.csv', index=False)

In [ ]:
results = (y_pred_lgb + y_xgb_pred + cat_pred) / 3 

output = pd.DataFrame({"id":test_data.id, "target":results})
output.to_csv('submission.csv', index=False)

In [ ]:
output